---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [3]:
def answer_one():
    
    number_of_spam_objects = spam_data[spam_data['target'] == 1].shape[0]
    number_of_all_objects = spam_data.shape[0]
    
    return (number_of_spam_objects/number_of_all_objects)*100

In [4]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer().fit(X_train)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    vect = CountVectorizer().fit(X_train)
    
    return max(vect.vocabulary_.keys(), key = len)

In [7]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [8]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    from sklearn.feature_extraction.text import CountVectorizer
    
    vect = CountVectorizer().fit(X_train)
    
    transformed_X_train = vect.transform(X_train)
    transformed_X_test = vect.transform(X_test)
    
    clf = MultinomialNB(alpha = 0.1)
    clf.fit(transformed_X_train, y_train)
    
    return roc_auc_score(y_test, clf.predict(transformed_X_test))

In [9]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    tifd_vect = TfidfVectorizer().fit(X_train)
    transformed_train_set = tifd_vect.transform(X_train) # transformed X_train set to features where each feature is a token of the 
                                                         # of the entire X_train. Displayed in term-document matrix form:
                                                         # ((row index, feature index/col index), tf-idf value)

    # feature names of the transformed X_train data
    feature_names = np.array(tifd_vect.get_feature_names())

    # Get top 20 features with largest tf-idf

    transformed_train_set.max(0) # return the biggest tf-idf score for each feature (i.e. return max values along axis = 0 or 
                                # columns) of the transformed data
    max_tfidf_values = transformed_train_set.max(0).toarray()[0] 
                                 # Change to array form with dimension 1. I.e. the first value of this array is 
                                 # feature 1 and its corresponding max tf-idf value, the second value is feature 2 and its 
                                 # corresponding max tf-idf value, etc..
    transformed_train_set.max(0).toarray()[0].argsort() 
                                            # return the array of index of the tf-idf values, where this array of index is sorted 
                                            # in ascending order of tf-idf values. I.e. Sort array of tf-idf values in ascending
                                            # order and replace the tf-idf values with its the arguments (index are the arguments). 

    top_20_tfidf_index = max_tfidf_values.argsort()[-1:-21:-1]
    top_20_tfidf_feature_names = [feature_names[x] for x in top_20_tfidf_index]
    top_20_tfidf_values = [max_tfidf_values[x] for x in top_20_tfidf_index]

    top_20_tfidf = list(zip(top_20_tfidf_feature_names, top_20_tfidf_values))
    top_20_tfidf_alphabetically = sorted(top_20_tfidf, key = lambda x: x[0])

    # Get bottom 20 features with smallest tf-idf

    bottom_20_tfidf_index = max_tfidf_values.argsort()[:20]
    bottom_20_tfidf_feature_names = [feature_names[x] for x in bottom_20_tfidf_index]
    bottom_20_tfidf_values = [max_tfidf_values[x] for x in bottom_20_tfidf_index]

    bottom_20_tfidf = list(zip(bottom_20_tfidf_feature_names, bottom_20_tfidf_values))
    bottom_20_tfidf_alphabetically = sorted(bottom_20_tfidf, key = lambda x: x[0])

    # In pd series form

    idx, values = zip(*top_20_tfidf_alphabetically)
    top_20_tfidf_series = pd.Series(values, idx)
    
    idx, values = zip(*bottom_20_tfidf_alphabetically)
    bottom_20_tfidf_series = pd.Series(values, idx)

    return (bottom_20_tfidf_series, top_20_tfidf_series)

In [11]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 diwali          0.091250
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 mornings        0.091250
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 blank        0.932702
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 tick         0.980166
 too          1.000000
 where        1.000000
 yup          1.000000
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [12]:
def answer_five():
    # Fit and transform X_train using Tfidf vectorizer that returns feature terms which have document frequency > 3 (i.e. minimum
    # document frequency = 3)

    from sklearn.feature_extraction.text import TfidfVectorizer

    tifd_vect = TfidfVectorizer(min_df = 3).fit(X_train) # document frequency: the count of documents that contain a particular
                                                         # term at least once. E.g. [doc1: 'a', doc2: 'a b', doc3: 'b b b b', 
                                                         # doc4: 'a c c', 'd']
                                                         # then document frequency of 'a' is 3, df of 'b' is 2, df of 'c' is 1, and
                                                         # df of 'd' is 1. 
    transformed_train_set = tifd_vect.transform(X_train)


    # Fit multinomial naive bayes classifier model with alpha = 0.1, then compute AUC score using transformed test data

    from sklearn.naive_bayes import MultinomialNB

    clf = MultinomialNB(alpha = 0.1).fit(transformed_train_set, y_train)

    from sklearn.metrics import roc_auc_score

    transformed_test_set = tifd_vect.transform(X_test)

    return roc_auc_score(y_test, clf.predict(transformed_test_set))

In [13]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [14]:
def answer_six():
    
    spam_data['document_length'] = [len(x) for x in spam_data['text']]
    target_groupby_apply_mean = spam_data.groupby(['target']).mean()
    
    average_length_not_spam= target_groupby_apply_mean.loc[0]
    average_length_spam = target_groupby_apply_mean.loc[1]
    
    return (average_length_not_spam[0], average_length_spam[0])

In [15]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [16]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [17]:
# Fit and transform X_train using Tfidf vectorizer that returns feature terms which have document frequency > 5 (i.e. minimum
# document frequency = 5)

from sklearn.feature_extraction.text import TfidfVectorizer

tifd_vect = TfidfVectorizer(min_df = 5).fit(X_train) 
transformed_train_set = tifd_vect.transform(X_train)

# add an additional feature (the length of document (number of characters))to this document-term matrix

length_of_document_train = [len(x) for x in X_train]
feature_added_to_transformed_train = add_feature(transformed_train_set, length_of_document_train) 
                                                                                    # length_of_document is the 1469th 
                                                                                   # feature with column index 1468

from sklearn.svm import SVC
# Fit SVM model (support vector classification model) with C = 10000, and transform test data

clf = SVC(C = 10000).fit(feature_added_to_transformed_train, y_train)

# Compute area under curve AUC

from sklearn.metrics import roc_auc_score

transformed_test_set = tifd_vect.transform(X_test)
length_of_document_test = [len(x) for x in X_test]
feature_added_to_transformed_test = add_feature(transformed_test_set, length_of_document_test)

roc_auc_score(y_test, clf.predict(feature_added_to_transformed_test))

0.95813668234215565

In [18]:
from sklearn.svm import SVC

def answer_seven():
# Fit and transform X_train using Tfidf vectorizer that returns feature terms which have document frequency > 5 (i.e. minimum
# document frequency = 5)

    from sklearn.feature_extraction.text import TfidfVectorizer

    tifd_vect = TfidfVectorizer(min_df = 5).fit(X_train) 
    transformed_train_set = tifd_vect.transform(X_train)

    # add an additional feature (the length of document (number of characters))to this document-term matrix

    length_of_document_train = [len(x) for x in X_train]
    feature_added_to_transformed_train = add_feature(transformed_train_set, length_of_document_train) 
                                                                                        # length_of_document is the 1469th 
                                                                                       # feature with column index 1468

    # Fit SVM model (support vector classification model) with C = 10000, and transform test data

    clf = SVC(C = 10000).fit(feature_added_to_transformed_train, y_train)

    # Compute area under curve AUC

    from sklearn.metrics import roc_auc_score

    transformed_test_set = tifd_vect.transform(X_test)
    length_of_document_test = [len(x) for x in X_test]
    feature_added_to_transformed_test = add_feature(transformed_test_set, length_of_document_test)
    
    return roc_auc_score(y_test, clf.predict(feature_added_to_transformed_test))

In [19]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [20]:
def answer_eight():
    
    import re

    spam_data['list_of_digits'] = spam_data['text'].apply(lambda x: ''.join(re.findall(r'[0-9]', x)))
    spam_data['number_of_digits'] = [len(x) for x in spam_data['list_of_digits']]
    target_groupby_apply_mean = spam_data.groupby(['target']).mean()
    target_groupby_apply_mean['number_of_digits']

    average_num_digits_not_spam = target_groupby_apply_mean['number_of_digits'][0]
    average_num_digits_spam = target_groupby_apply_mean['number_of_digits'][1]

    
    return (average_num_digits_not_spam, average_num_digits_spam)

In [21]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [22]:
from sklearn.linear_model import LogisticRegression

def answer_nine():

    # Fit and transform X_train
    
    from sklearn.feature_extraction.text import TfidfVectorizer
    tifd_vect = TfidfVectorizer(min_df = 5, ngram_range = (1,3)).fit(X_train) 
                                                                        # The decision rule for partitioning a text document
                                                                        # into tokens of variyng n-grams is not explained 
                                                                        # from the Python description, but it is likely
                                                                        # determined based on Tfidf scores of terms, and 
                                                                        # combining low Tfidf score terms (e.g. 'the', 'a',
                                                                        # 'to', 'this', 'which', etc.) with high Tfidf score
                                                                        # terms (i.e. words that are less common and thus 
                                                                        # more significant in its impact on the document).

    transformed_train = tifd_vect.transform(X_train)

    # Add additional features to transformed train set. 

    train_dataframe = X_train.to_frame()
    train_dataframe_plus_features = train_dataframe.merge(spam_data, how = 'left', left_index = True, right_index = True)
                # Merge X_train with spam_data on left merge (i.e. perform left merge on the index of X_train and index of
                # spam_data)

    length_of_document_train = [x for x in train_dataframe_plus_features['document_length']] 
                                                                                        # feature to add to transformed train
    transformed_train_plus_additional_features = add_feature(transformed_train, length_of_document_train)

    number_of_digits_train = [x for x in train_dataframe_plus_features['number_of_digits']] # another feature to add to 
                                                                                            # train transformed.

    transformed_train_plus_additional_feaures = add_feature(transformed_train_plus_additional_features, number_of_digits_train)

    # Fit a Logistic Regression model with regularization C=100. 
    # Then compute the area under the curve (AUC) score using the transformed test data.

    regressor = LogisticRegression(C = 100).fit(transformed_train_plus_additional_feaures, y_train)
    
    # Calculate AUC score
    
    from sklearn.metrics import roc_auc_score

    transformed_test = tifd_vect.transform(X_test)
    test_dataframe = X_test.to_frame()
    test_dataframe_plus_features = test_dataframe.merge(spam_data, how = 'left', left_index = True, right_index = True)
    length_of_document_test = [x for x in test_dataframe_plus_features['document_length']] 
    transformed_test_plus_additional_features = add_feature(transformed_test, length_of_document_test)
    number_of_digits_test = [x for x in test_dataframe_plus_features['number_of_digits']]

    transformed_test_plus_additional_features = add_feature(transformed_test_plus_additional_features, number_of_digits_test)

    return roc_auc_score(y_test, regressor.predict(transformed_test_plus_additional_features))

In [23]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [24]:
def answer_ten():
    
    import re

    # Regex pattern to use example: re.findall(r'\W(?=\w)|\W(?!\w)', '1a2b3A-c_?d_ag') # or re.findall(r'\W', '1a2b3A-c_?d_ag')

    # Note: Just to clarify how (...) open brackets work with | operator, the syntax inside () are considered a distinct group
    # of the entire regex syntax. 
    # e.g. For re.findall(r'(\w(?=\w))|(\w(?!\w))', '1a2b3A-c_?d_ag') (\W(?=\w)) and (\W(?!\w) are the two groups of this 
    # entire regex pattern.

    spam_data['non_word_characters'] = spam_data['text'].apply(lambda x: re.findall(r'\W(?=\w)|\W(?!\w)', x))

    spam_data['number_of_non_word_characters'] = [len(x) for x in spam_data['non_word_characters']]

    average_number_of_non_word_characters_not_spam = spam_data.groupby('target').mean()['number_of_non_word_characters'][0]
    average_number_of_non_word_characters_spam = spam_data.groupby('target').mean()['number_of_non_word_characters'][1]
    
    return (average_number_of_non_word_characters_not_spam, average_number_of_non_word_characters_spam)

In [25]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [26]:
def answer_eleven():
    
    # Fit and transform X_train using Count Vectorizer under following conditions: (1) min_df = 5, ngram_range = (2,5)

    from sklearn.feature_extraction.text import CountVectorizer

    vect = CountVectorizer(min_df = 5, ngram_range = (2,5), analyzer = 'char_wb').fit(X_train)
    transformed_train = vect.transform(X_train)
    transformed_test = vect.transform(X_test)

    # Add three additional features to transformed_train document-term matrix. Repeat the same process for transformed_test.

    train_dataframe = X_train.to_frame()
    train_dataframe_plus_features = train_dataframe.merge(spam_data, how = 'left', left_index = True, right_index = True)

    length_of_document_train = [x for x in train_dataframe_plus_features['document_length']]
    number_of_digits_train = [x for x in train_dataframe_plus_features['number_of_digits']]
    number_of_non_word_characters_train = [x for x in train_dataframe_plus_features['number_of_non_word_characters']]

    transformed_train_plus_additional_features = add_feature(transformed_train, [length_of_document_train, number_of_digits_train,
                                                                                 number_of_non_word_characters_train])

    test_dataframe = X_test.to_frame()
    test_dataframe_plus_features = test_dataframe.merge(spam_data, how = 'left', left_index = True, right_index = True)

    length_of_document_test = [x for x in test_dataframe_plus_features['document_length']]
    number_of_digits_test = [x for x in test_dataframe_plus_features['number_of_digits']]
    number_of_non_word_characters_test = [x for x in test_dataframe_plus_features['number_of_non_word_characters']]

    transformed_test_plus_additional_features = add_feature(transformed_test, [length_of_document_test, number_of_digits_test,
                                                                                 number_of_non_word_characters_test])

    # Fit a logistic regression model with C = 100.

    from sklearn.linear_model import LogisticRegression

    regressor = LogisticRegression(C = 100).fit(transformed_train_plus_additional_features, y_train)

    # Compute AUC score using transformed test data.

    from sklearn.metrics import roc_auc_score

    auc_score = roc_auc_score(y_test, regressor.predict(transformed_test_plus_additional_features))

    # find the 10 smallest and 10 largest coefficients from the model

    model_coefficients = regressor.coef_[0]
    feature_names = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])

    top_10_smallest_coef_feature_index = model_coefficients.argsort()[0:10]
    top_10_biggest_coef_feature_index = model_coefficients.argsort()[-1:-11:-1]

    top_10_smallest_coef_feature_names = [feature_names[x] for x in top_10_smallest_coef_feature_index]
    top_10_biggest_coef_feature_names =[feature_names[x] for x in top_10_biggest_coef_feature_index]

    return (auc_score, top_10_smallest_coef_feature_names, top_10_biggest_coef_feature_names)

In [27]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar'])